# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [14]:
vacation_df = pd.read_csv('../WeatherPy\output_data/WeatherPy.csv')
del vacation_df['Unnamed: 0']
vacation_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,dikson,73.5069,80.5464,-14.93,97.0,86.0,4.32,RU,1.619819e+09
1,assiniboia,49.6334,-105.9845,25.00,14.0,100.0,9.77,CA,1.619819e+09
2,jamestown,42.0970,-79.2353,6.11,56.0,90.0,12.35,US,1.619819e+09
3,tutoia,-2.7619,-42.2744,25.57,86.0,91.0,2.86,BR,1.619819e+09
4,karratha,-20.7377,116.8463,26.47,43.0,4.0,2.50,AU,1.619819e+09
...,...,...,...,...,...,...,...,...,...
597,terrace,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
598,turayf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
599,gulu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
600,ahar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
vacation_df.dropna()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,dikson,73.5069,80.5464,-14.93,97.0,86.0,4.32,RU,1.619819e+09
1,assiniboia,49.6334,-105.9845,25.00,14.0,100.0,9.77,CA,1.619819e+09
2,jamestown,42.0970,-79.2353,6.11,56.0,90.0,12.35,US,1.619819e+09
3,tutoia,-2.7619,-42.2744,25.57,86.0,91.0,2.86,BR,1.619819e+09
4,karratha,-20.7377,116.8463,26.47,43.0,4.0,2.50,AU,1.619819e+09
...,...,...,...,...,...,...,...,...,...
549,bandarbeyla,54.5163,-128.6035,9.00,81.0,90.0,9.77,CA,1.619821e+09
550,pitea,31.6725,38.6637,25.00,23.0,40.0,7.20,SA,1.619821e+09
551,mandalgovi,2.7746,32.2990,18.41,94.0,86.0,0.79,UG,1.619820e+09
552,cassilandia,38.4774,47.0699,12.39,86.0,100.0,2.79,IR,1.619821e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
gmaps.configure(api_key=g_key)
locations = vacation_df[["Lat","Lng"]]
humidity = vacation_df["Humidity"].astype(float)

In [18]:
locations = locations.dropna()
humidity = humidity.dropna()

In [34]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [39]:
#Narrow down the DataFrame to find your ideal weather condition. For example:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
#Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

ideal_weather = vacation_df.loc[(vacation_df["Wind Speed"] < 10) &
                                (vacation_df["Cloudiness"] == 0) &
                                (vacation_df["Max Temp"] >= 25) & 
                                (vacation_df["Max Temp"] <= 30)]

ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
50,belushya guba,25.2867,51.5333,28.00,57.0,0.0,3.60,QA,1.619819e+09
96,nelson bay,13.9553,7.6712,27.72,6.0,0.0,2.34,NE,1.619819e+09
147,imeni poliny osipenko,26.3260,43.9750,25.00,27.0,0.0,2.06,SA,1.619819e+09
187,kemijarvi,-20.6264,-49.6492,27.00,41.0,0.0,2.57,BR,1.619820e+09
278,pangnirtung,27.9769,-114.0611,29.33,25.0,0.0,6.87,MX,1.619820e+09
294,vaitupu,25.0213,37.2685,29.67,35.0,0.0,1.97,SA,1.619820e+09
372,jamame,4.9333,-52.3333,26.11,83.0,0.0,2.06,GF,1.619820e+09
428,verkhoyansk,-8.4753,-74.8058,28.00,47.0,0.0,4.12,PE,1.619820e+09
468,palmares do sul,19.3167,95.1833,27.30,71.0,0.0,1.06,MM,1.619820e+09
469,sistranda,4.7263,-52.3274,26.11,83.0,0.0,1.80,GF,1.619820e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [40]:
hotel_df = ideal_weather
hotel_df["Hotel Name"] = ""
hotel_df

<ipython-input-40-29a6706857e5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
50,belushya guba,25.2867,51.5333,28.00,57.0,0.0,3.60,QA,1.619819e+09,
96,nelson bay,13.9553,7.6712,27.72,6.0,0.0,2.34,NE,1.619819e+09,
147,imeni poliny osipenko,26.3260,43.9750,25.00,27.0,0.0,2.06,SA,1.619819e+09,
187,kemijarvi,-20.6264,-49.6492,27.00,41.0,0.0,2.57,BR,1.619820e+09,
278,pangnirtung,27.9769,-114.0611,29.33,25.0,0.0,6.87,MX,1.619820e+09,
294,vaitupu,25.0213,37.2685,29.67,35.0,0.0,1.97,SA,1.619820e+09,
372,jamame,4.9333,-52.3333,26.11,83.0,0.0,2.06,GF,1.619820e+09,
428,verkhoyansk,-8.4753,-74.8058,28.00,47.0,0.0,4.12,PE,1.619820e+09,
468,palmares do sul,19.3167,95.1833,27.30,71.0,0.0,1.06,MM,1.619820e+09,
469,sistranda,4.7263,-52.3274,26.11,83.0,0.0,1.80,GF,1.619820e+09,


In [43]:
# find the closest restaurant of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "hotel",
    "key": g_key,
    "keyword" : "hotel"
    }
print("----------------------------------------------------------")
print("Search started, please wait. ")
print("----------------------------------------------------------")

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    
    params["location"] = f"{lat},{lng}"
  
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest hotel in {city} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("----------------------------------------------------------")
print("----------------------------------------------------------")
print("Search completed successfully")
print("----------------------------------------------------------")
hotel_df

----------------------------------------------------------
Search started, please wait. 
----------------------------------------------------------
Retrieving Results for Index 50: belushya guba.
Closest hotel in belushya guba is The Curve Hotel.
----------------------------------------------------------
Retrieving Results for Index 96: nelson bay.


C:\Users\joses\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Missing field/result... skipping.
----------------------------------------------------------
Retrieving Results for Index 147: imeni poliny osipenko.
Closest hotel in imeni poliny osipenko is Mövenpick Hotel Qassim.
----------------------------------------------------------
Retrieving Results for Index 187: kemijarvi.
Closest hotel in kemijarvi is Klinn Hotel.
----------------------------------------------------------
Retrieving Results for Index 278: pangnirtung.
Closest hotel in pangnirtung is Hotel Los Caracoles.
----------------------------------------------------------
Retrieving Results for Index 294: vaitupu.
Closest hotel in vaitupu is HP Red Sea Hotel.
----------------------------------------------------------
Retrieving Results for Index 372: jamame.
Closest hotel in jamame is Mercure Cayenne Royal Amazonia.
----------------------------------------------------------
Retrieving Results for Index 428: verkhoyansk.
Missing field/result... skipping.
------------------------------

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
50,belushya guba,25.2867,51.5333,28.00,57.0,0.0,3.60,QA,1.619819e+09,The Curve Hotel
96,nelson bay,13.9553,7.6712,27.72,6.0,0.0,2.34,NE,1.619819e+09,
147,imeni poliny osipenko,26.3260,43.9750,25.00,27.0,0.0,2.06,SA,1.619819e+09,Mövenpick Hotel Qassim
187,kemijarvi,-20.6264,-49.6492,27.00,41.0,0.0,2.57,BR,1.619820e+09,Klinn Hotel
278,pangnirtung,27.9769,-114.0611,29.33,25.0,0.0,6.87,MX,1.619820e+09,Hotel Los Caracoles
294,vaitupu,25.0213,37.2685,29.67,35.0,0.0,1.97,SA,1.619820e+09,HP Red Sea Hotel
372,jamame,4.9333,-52.3333,26.11,83.0,0.0,2.06,GF,1.619820e+09,Mercure Cayenne Royal Amazonia
428,verkhoyansk,-8.4753,-74.8058,28.00,47.0,0.0,4.12,PE,1.619820e+09,
468,palmares do sul,19.3167,95.1833,27.30,71.0,0.0,1.06,MM,1.619820e+09,KBZ Bank
469,sistranda,4.7263,-52.3274,26.11,83.0,0.0,1.80,GF,1.619820e+09,Amazone Lodges Oyack


In [44]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [45]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))